In [1]:
from bls_py import bls, ec
from random import randint, seed

We will work in the G1 (Fq) group since public keys are in the G1 group.

In [2]:
G = ec.generator_Fq() # G1
l = ec.default_ec.n   # the G1 group order

# Pederson Commitment prep:

gamma = randint(1, l) # It is assumed that gamma is unknown to *anyone* (otherwise our commitments can be tampered with)
H = gamma * G # commit(a, x) = xG + aH

In [3]:
def Hp(point: bls.AffinePoint) -> bls.AffinePoint:
    """Hash a point to point"""
    return ec.hash_to_point_Fq(point.x.serialize() + point.y.serialize())

def Hs(*xs) -> int:
    """Hash to field, result will be an integer in 0..l"""
    h = b''
    for x in xs:
        if type(x) == str:
            b = bytes(x, encoding="utf-8")
        elif type(x) == bls.AffinePoint:
            b = x.serialize()
        elif type(x) == int:
            b = x.to_bytes(256, "big")
        elif type(x) in [list, tuple]:
            b = b''
            for xi in x:
                b += Hs(xi).to_bytes(256, "big")
        else:
            raise Exception("BAD VALUE", x)

        h = ec.hash256(h + b)

    return int.from_bytes(h, "big") % l
    
def rand_Z():
    return randint(1, l)

def rand_P():
    return rand_Z() * G
    
def commit(value, mask):
    """Pederson Commitment to `value` masked by `mask`"""
    return mask * G + value * H

In [12]:
seed(2)

N = 10 # ring size
M = 3 # number of inputs

msg = "hello" # in practice this is the hash of all transaction data besides the ring signature

# input amounts
in_a = [randint(1, 4) for _ in range(M)]
in_blinding = [rand_Z() for _ in in_a]

# output amounts
out_a = [sum(in_a)] # TODO: deal with multiple outputs
out_P = [rand_Z() * G for _ in out_a]

assert sum(in_a) - sum(out_a) == 0

# Real Keys
x_pi = [rand_Z() for _ in range(M)]
P_pi = [x * G for x in x_pi]

# Decoy Keys
P = [[rand_Z() * G for _ in range(M)] for _ in range(N - 1)]

# Commitments from each decoy input
C = [[commit(rand_Z(), rand_Z()) for _ in range(M)] for _ in range(N - 1)]

pi = randint(0, N-1)
print("pi =", pi)

P.insert(pi, P_pi)
C.insert(pi, [commit(in_a[m], in_blinding[m]) for m in range(M)])

pi = 3


In [19]:
# Used for generating pseudo commitments to zero
pseudo_blinding = [rand_Z() for _ in range(M)]
pseudo_C = [commit(in_a[m], pseudo_blinding[m]) for m in range(M)]

out_blinding = [rand_Z() for _ in range(len(out_a) - 1)]
out_blinding.append((sum(pseudo_blinding) - sum(out_blinding)) % l)
out_C = [commit(out_a[i], out_blinding[i]) for i in range(len(out_a))]

assert sum(pseudo_C) == sum(out_C)


hidden_C = [[C[n][m] - pseudo_C[m] for m in range(M)] for n in range(N)]

# prepare Rings for each input
R = [[(P[n][m], hidden_C[n][m]) for n in range(N)] for m in range(M)]

### Perform MLSAG on $R$

Since we don't want Key Images for the hidden_C part of R, we have to modify MLSAG slightly when dealing with the second column of R to avoid key images.

In [23]:
# Key Images
I = [x_pi[m] * Hp(P[pi][m]) for m in range(M)]
c = [None] * M
r = [None] * M
for m in range(M):
    x = [x_pi[m], (in_blinding[m] - pseudo_blinding[m] ) % l]
    alpha = [rand_Z(), rand_Z()]
    r[m] = [[rand_Z(), rand_Z()] for n in range(N)]
    r[m][pi] = [None, None]

    c[m] = [None] * N
    c[m][(pi + 1) % N] = Hs(
        msg,
        *[alpha[0] * G, alpha[1] * G],
        # We exclude the `r[n][1] * ..` term to avoid dependence on the pseudo commitment key image
        *[alpha[0] * Hp(R[m][pi][0])]
    )

    for offset in range(1, N):
        n = (pi + offset) % N
        c[m][(n + 1) % N] = Hs(
            msg,
            *[r[m][n][j] * G + c[m][n] * R[m][n][j] for j in range(2)],
            *[r[m][n][0] * Hp(R[m][n][0]) + c[m][n] * I[m]] 
        )
        
    # Correct r[pi] using r_pi_j = alpha_j - c_pi * x_pi_j mod l
    r[m][pi] = [(alpha[j] - c[m][pi] * x[j]) % l for j in range(2)]
    
    
    # For our sanity, check a few identities
    for j in range(2):
        assert x[j] * G == R[m][pi][j], f"{j}: {x[j] * G} != {R[m][pi][j]}"
        assert (alpha[j] - c[m][pi]*x[j]) % l * G == alpha[j] * G - (c[m][pi] * x[j]) * G
        assert (alpha[j] - c[m][pi]*x[j]) % l * G + c[m][pi] * R[m][pi][j] == alpha[j] * G
        assert r[m][pi][j] * G + c[m][pi]*R[m][pi][j] == (alpha[j] - c[m][pi]*x[j]) % l * G + c[m][pi] * R[m][pi][j]
        assert r[m][pi][j] * Hp(R[m][pi][j]) + c[m][pi] * I[m] == (alpha[j] - c[m][pi]*x[j]) % l * Hp(R[m][pi][j]) + c[m][pi] * I[m]
        if j == 0:
            assert x[j] * Hp(R[m][pi][j]) == I[m]
            assert (-c[m][pi] % l) * x[j] * Hp(R[m][pi][j]) == (-c[m][pi] % l) * I[m]
            assert (alpha[j] - c[m][pi] * x[j]) % l * Hp(R[m][pi][j]) == alpha[j] * Hp(R[m][pi][j]) - c[m][pi] * I[m]
            assert alpha[j] * Hp(R[m][pi][j]) - c[m][pi] * I[m] == alpha[j] * Hp(R[m][pi][j]) - c[m][pi] * x[j] * Hp(R[m][pi][j])
        assert r[m][pi][j] * Hp(R[m][pi][j]) + c[m][pi] * I[m] == (alpha[j] - c[m][pi] * x[j]) % l * Hp(R[m][pi][j]) + c[m][pi] * I[m]

sig = ([c[m][0] for m in range(M)], r)

In [24]:
tx = {
    "input": {
        "ring_members": P,
        "mlsag_sigs": sig,
        "key_images": I,
        "pseudo_commitments": pseudo_C,
    },
    "output": {
        "owners": out_P,
        "commitments": out_C,
        "range_proofs": None, # TODO
    }
}

In [26]:
# Verification

# Verify Key Images are in G
for m in range(M):
    assert l * I[m] == 0 * G

# Verify MLSAGs
for m in range(M):
    cprime = [None] * N
    cprime[0] = c[m][0]

    for n in range(N):
        cprime[(n + 1) % N] = Hs(
            msg,
            *[r[m][n][j] * G + cprime[n] * R[m][n][j] for j in range(2)],
            *[r[m][n][0] * Hp(R[m][n][0]) + cprime[n] * I[m]]
        )

    assert c[m][0] == cprime[0]
    
# Verify pseudo commitments sum to output commitments
assert sum(tx["input"]["pseudo_commitments"]) == sum(tx["output"]["commitments"])

print("all checks out")

all checks out
